# Implement a RAG pipeline with a Stuff Documents chain


## Load Modules


In [7]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import BaseMessage
from pathlib import Path

## Defining Functions for Chain Invocation with Memory


In [8]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationBufferMemory()


def load_memory(_) -> list[BaseMessage]:
    return memory.chat_memory.messages


def invoke_chain(question: str) -> None:
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})
    print(result.content)

## Implement RAG with Memory


In [9]:
cache_path = Path("../.cache").resolve()
cache_dir = LocalFileStore(cache_path)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=600,
    chunk_overlap=100,
    separator="\n",
)

loader = TextLoader("../files/1984.txt")

docs = loader.load_and_split(text_splitter=splitter)

embedding = OpenAIEmbeddings()

cached_embedding = CacheBackedEmbeddings.from_bytes_store(embedding, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embedding)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": RunnableLambda(load_memory),
    }
    | prompt
    | llm
)

## Invoke Chains


In [10]:
invoke_chain("Is Aaronson guilty?")

I don't know.


In [11]:
invoke_chain("What message did he write in the table?")

He wrote "FREEDOM IS SLAVERY" in large clumsy capitals.


In [12]:
invoke_chain("Who is Julia?")

Julia is a character mentioned in the text.
